<a href="https://colab.research.google.com/github/danibosch/mentoria-diplo-datos2021/blob/main/Cintelink_AyVdD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis y Visualización de Datos 

In [ ]:
import io
import matplotlib
import matplotlib.pyplot as plt
import numpy
import pandas as pd
import seaborn

seaborn.set_context('talk')
# Set float format
pd.set_option('display.float_format','{:.2f}'.format)

# Set style
seaborn.set_style("darkgrid")
seaborn.set_palette('pastel')
seaborn.set_context("paper", rc={"font.size":12,"axes.titlesize":12,"axes.labelsize":12}) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def convert2float32(n):
    try:
        return numpy.float32(n)
    except:
        return numpy.nan

def convert2float16(n):
    try:
        return numpy.float16(n)
    except:
        return numpy.nan

def convert2int16(n):
    try:
        return numpy.int16(n)
    except:
        return numpy.nan

def convert2int8(n):
    try:
        return numpy.int8(n)
    except:
        return numpy.nan

In [ ]:
dtypes = {
    "id": "category",
    "id_equipo": "category",
    "id_tanque": "category",
    "producto": "category",
    "id_empresa": "category",
    "id_canal": "category",
    "nombre_producto": "category",
    "industria": "category",
    "alarma": "boolean"
}

# Para evitar datos erróneos en el parsing
converters = {
    "id_industria": convert2float16,
    "volumen": convert2float32,
    "vbat1": convert2int16,
    "vbat2": convert2int16,
    "capacidad": convert2float32,
    "fuel_level_dmm": convert2float32,
    "water_level_dmm": convert2float32,
    "water_volume_lts": convert2float32,
    "temp5": convert2float16,
    "temp4": convert2float16,
    "temp3": convert2float16,
    "temp2": convert2float16,
    "temp1": convert2float16,
    "temperatura": convert2float16,
    "coef_var_vol": convert2float16,
    "density": convert2float32
}

In [ ]:
# filename = '/content/drive/MyDrive/Colab Notebooks/DiploDatos/Mentoría/Datasets/dataset100mil.csv'
filename = '/content/drive/MyDrive/Colab Notebooks/DiploDatos/Mentoría/Datasets/StorageInventory_2021_Q1.csv'
raw_df = pd.read_csv(filename, converters=converters, dtype=dtypes, parse_dates=["timestamp"])

In [ ]:
raw_df.info(memory_usage="deep")

In [ ]:
raw_df.head()

In [ ]:
# Tamaño del dataset
len(raw_df)

## Limpieza de datos

### Tipo de datos de columnas y separación de algunos datos

In [ ]:
raw_df['volumen'] = raw_df['volumen'].astype(numpy.float32)
raw_df['temperatura'] = raw_df['temperatura'].astype(numpy.float16)
raw_df['temp5'] = raw_df['temp5'].astype(numpy.float16)
raw_df['temp4'] = raw_df['temp4'].astype(numpy.float16)
raw_df['temp3'] = raw_df['temp3'].astype(numpy.float16)
raw_df['temp2'] = raw_df['temp2'].astype(numpy.float16)
raw_df['temp1'] = raw_df['temp1'].astype(numpy.float16)
raw_df['fuel_level_dmm'] = raw_df['fuel_level_dmm'].astype(numpy.float32)
raw_df['water_level_dmm'] = raw_df['water_level_dmm'].astype(numpy.float32)
raw_df['water_volume_lts'] = raw_df['water_volume_lts'].astype(numpy.float32)
raw_df['capacidad'] = raw_df['capacidad'].astype(numpy.float32)
raw_df['coef_var_vol'] = raw_df['coef_var_vol'].astype(numpy.float16)
raw_df['density'] = raw_df['density'].astype(numpy.float16)
raw_df['vbat1'] = raw_df['vbat1'].astype(numpy.int16)
raw_df['vbat2'] = raw_df['vbat2'].astype(numpy.int16)
raw_df['id_industria'] = raw_df['id_industria'].astype("category")
raw_df['codigo'] = raw_df['codigo'].astype("str")

In [ ]:
raw_df.info(memory_usage="deep")

La columna `código` contiene dos dígitos, de los cuales, el primero representa el estado y el segundo representa la cantidad de ecos enviados por la sonda.

In [ ]:
raw_df['codigo'].unique()

In [ ]:
# Trabajamos sobre una copia y no sobre el original
df = raw_df.copy()

In [ ]:
# Si se llena la memoria, ejecutar
del raw_df

In [ ]:
# Separamos el código en dos datos
df['c'] = df['codigo'].apply(lambda x: str(x)[0])
df['echoes'] = pd.to_numeric(df['codigo'].apply(lambda x: str(x)[1]), errors='coerce')

### Valores faltantes

Las columnas `volumen` y `temperatura`

#### Volúmenes nulos

In [ ]:
nan = numpy.nan
df.query("volumen == @nan")

#### Temperaturas nulas

In [ ]:
nan = numpy.nan
df.query("temperatura == @nan")

### Fuel level codi nan

In [ ]:
#análisis de intersección nan entre variables
df[df['temperatura'].isna()&df['codigo'].isna()&~df['fuel_level_dmm'].isna()]

### Eliminación de outliers

#### Condiciones de filtrado
A partir de la exploración de los datos definimos algunos criterios para el filtrado de nuestra base de datos. Por un lado, pudimos observar que la variable `volumen` posee valores mínimos negativos y que su máximo valor sobrepasa el valor máximo de la capacidad de los tanques, por lo que difinimos la primera y segunda condición de filtrado (cond_1 y cond_2) y nos quedamos con valores de volúmenes mayores o iguales a cero y menores a la capacidad máxima. Por otro lado, vimos que cuando la carga de alguna de las dos baterías es cero el nivel de combustible también es cero o produce valores nulos (nan) por lo que no se estaría estimando correctamente el valor de volumen, definiendo de esta manera las condidiones 3 y 4 (cond_3 y cond_4). Por último, no tuvimos en cuenta los valores nulos de código porque inferimos que el sensor no está funcionando y esto coincide con la existencia de valures nulos de temperatura y de nivel de combustible (cond_5).

In [ ]:
# Volúmenes negativos
cond_1 = df["volumen"] >= 0

# Volúmenes mayores a la capacidad del tanque
cond_2 = df["volumen"] < df["capacidad"].max()

# Voltaje del pulso de eco 0
cond_3 = df["vbat1"] != 0

# Batería con voltaje 0
cond_4 = df["vbat2"] != 0

# Esto lo pondría arriba, con los valores nulos
cond_5 = ~df['codigo'].isna()

In [ ]:
df_fil = df[cond_1 & cond_2 & cond_3 & cond_4 & cond_5]
df_fil

In [ ]:
df_fil.info()

In [ ]:
df_fil.describe().round()

In [ ]:
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, figsize=(15,9),gridspec_kw={"height_ratios": (.15, .85)})
 
seaborn.boxplot(df_fil.volumen, ax=ax_box)
seaborn.distplot(df_fil.volumen, ax=ax_hist, kde=False, hist=False)
seaborn.distplot(df_fil.volumen, ax=ax_hist, kde=True, hist=True,norm_hist=False)

plt.axvline(df_fil.volumen.quantile(0.25))
plt.axvline(df_fil.volumen.quantile(0.50),color='g')
plt.axvline(df_fil.volumen.quantile(0.75))
plt.ticklabel_format(style='plain', axis='x') 
plt.axvline(df_fil.volumen.mean(),color='red') 
ax_box.set_title
plt.show()

Filtrado de outliers teniendo en cuenta el rango intercuartílico

In [ ]:
q1=df_fil.volumen.quantile(0.25)
q3=df_fil.volumen.quantile(0.75)
RI=q3-q1
min=q1-2.5*RI
max=q3+2.5*RI
print("Límite inferior =", min)
print("Límite superior =", max)

In [ ]:
df_fil_out=  df_fil[df_fil.volumen < max]
df_fil_out[:3]

Filtrado de outliers teniendo en cuenta el la desviación estándar

In [ ]:
vol_mean=df.volumen.mean()
vol_std=df.volumen.std()
min_s=vol_mean - 2.5 * vol_std
max_s=vol_mean + 2.5 * vol_std
print("Límite inferior =", min_s)
print("Límite superior =", max_s)

In [ ]:
df_fil_out=df_fil[df_fil.volumen < max_s]
df_fil_out[:3]

In [ ]:
f, (ax_box, ax_hist) = plt.subplots(2, sharex=True, figsize=(15,9),gridspec_kw={"height_ratios": (.15, .85)})
 
seaborn.boxplot(df_fil_out.volumen, ax=ax_box)
seaborn.distplot(df_fil_out.volumen, ax=ax_hist, kde=False, hist=False)
seaborn.distplot(df_fil_out.volumen, ax=ax_hist, kde=True, hist=True,norm_hist=False)

plt.axvline(df_fil_out.volumen.quantile(0.25))
plt.axvline(df_fil_out.volumen.quantile(0.50),color='g')
plt.axvline(df_fil_out.volumen.quantile(0.75))
plt.ticklabel_format(style='plain', axis='x') 
plt.axvline(df_fil_out.volumen.mean(),color='red') 
ax_box.set_title
plt.show()

#### Códigos de error

In [ ]:
# Códigos de error
# Incluyendo Ok o excluyendo error?
ok_codes = ['N', 'L', 'V', 'U', 'P', 'T', 'A', 'I', '0']
df.query("c not in @ok_codes").head()

In [ ]:
error_codes = ['m', 'M', 'F']
df.query("c in @error_codes").head()

In [ ]:
# Obtenemos el mismo resultado?
len(df.query("c not in @ok_codes")) == len(df.query("c in @error_codes"))

#### Ecos cero

In [ ]:

df.query("echoes == 0").head()

#### Pérdida de ecos en un mismo tanque

#### Filtrado por rango intercuartílico de mediana móvil

Calculamos la mediana móvil del `volumen` con distintas ventanas de tiempo.

In [ ]:
window = 5
df['moving_avg'] = df.groupby('id_tanque')['volumen'].transform(
    lambda x: x.rolling(window=window, min_periods=1, center=True).mean()
)

In [ ]:
fig = plt.figure(figsize=(15, 7))
seaborn.lineplot(
    data=tanque, 
    y='volumen', 
    x='timestamp' 
)
seaborn.lineplot(
    data=tanque, 
    y='moving_avg', 
    x='timestamp' 
)
seaborn.lineplot(
    data=tanque, 
    y='upper_bond', 
    x='timestamp' 
)
seaborn.lineplot(
    data=tanque, 
    y='lower_bond', 
    x='timestamp' 
)
plt.ticklabel_format(style='plain', axis='y')
seaborn.despine()

In [ ]:
fig = plt.figure(figsize=(15, 7))
seaborn.lineplot(
    data=tanque[1400:1500], 
    y='volumen', 
    x='timestamp' 
)
seaborn.lineplot(
    data=tanque[1400:1480], 
    y='moving_avg', 
    x='timestamp' 
)
seaborn.lineplot(
    data=tanque[1400:1480], 
    y='upper_bond', 
    x='timestamp' 
)
seaborn.lineplot(
    data=tanque[1400:1480], 
    y='lower_bond', 
    x='timestamp' 
)
plt.ticklabel_format(style='plain', axis='y')
seaborn.despine()

### Uniformidad de tiempo de los datos

### Resampling del timeseries
#### Bfill

In [ ]:
# 10 minutos
tanque_resampled = tanque.resample('10T').bfill()
tanque_resampled

In [ ]:
fig = plt.figure(figsize=(15, 7))
seaborn.lineplot(
    data=tanque_resampled[1400:1500], 
    y='volumen', 
    x='timestamp' 
)
plt.ticklabel_format(style='plain', axis='y')
seaborn.despine()

### Agrupación de inventarios

### Normalización de valores

Ideas: (esto es curación?) 
- Ajustar por temperatura.
- Restar volumen de agua.

## Análisis

Para esta etapa se procede a analizar los distintos inventarios agrupándolos por diferentes criterios.

### Invetarios sobre centros operativos

Se seleccionan algunos centros operativos para analizar el comportamiento de sus inventarios. Para esto, realizamos una selección aleatoria de la columna `id_equipo`.

### Patrones de manejo de inventario por industria

### Descriptores estadísticos de los inventarios

### Distribución de manejo de inventarios por industria

Graficamos un boxplot para poder visualizar la distribución de los volúmenes que maneja cada industria.

### Correlación de cada una de las features del dataset

### Primera y segunda derivada del `volumen` en el tiempo

La primer derivada nos da información de los movimientos del `volumen` en el tiempo. Si el líquido sube, la pendiente es positiva, si es negativa, el líquido baja. Valores cercanos a cero nos indican poco movimiento, mientras que valores más grandes nos indican que el líquido se mueve en grandes cantidades.

In [ ]:
# Graficar un tanque + diff

La segunda derivada nos puede indicar cuál es la aceleración del movimiento del tanque.

In [ ]:
# Graficar un tanque + diff de diff

### Consumos por unidad de tiempo por centro operativo

Se seleccionan algunos centros operativos

### Patrones de consumo por industria

### Análisis estadísitico de los consumos en general

### Análisis estadístico de los consumos por industria

### Media móvil de inventarios para cada indsutria

Se selecciona un centro operativo por industria para realizar el análisis.

In [ ]:
# Ventana numérica

In [ ]:
# Ventana por tiempo

### Media móvil de consumos para cada industria

In [ ]:
# Ventana numérica

In [ ]:
# Ventana de tiempo

## Conclusiones

Cómo harían un cálculo simple para estimar inventarios en el corto plazo?

Ideas:
- Repetir Último valor visto.
- Drift (media de últimos valores vistos)
- Última pendiente
- Media movil de últimas pendientes
- última pendiente, y aplicar última aceleración (d' y d'')
- Media movil de últimas pendientes y aplicar mm de última aceleración
- Conjunto de últimos valores + último valor 

### Repetición del último valor visto en el tiempo.

### Último valor visto y media móvil

## Notas

Ordenar:  
Limpieza
1. Valores faltantes
 - Ver volúmenes nulos si hay
 
2. Outliers
  - Volúmenes negativos (quitar)
  - Volúmenes por encima de la capacidad 
  - Volúmenes no numéricos
  - Tanques con pocos registros
  - Volúmenes que se salen de cierto rango alrededor de la media móvil (intervalos de confianza? porcentajes? ver histogramas?)
  - Registros con códigos de error (definidas en tabla)
  - Registros con ecos 0 (tener en cuenta sondas de presión)
  - Temp5 fuera de rango (idem anterior media movil? intervalos?)
  - vbat1/2 fuera de rango (idem anterior)
  - Capacidades negativas, muy chicas. Ver histograma

3. No se encuentran uniformes
  - Graficar un par donde se vean
  - Calcular diff de tiempos, distribucion.

4. Resample
  - interpolación (lineal? cúbica?)
  - con bfill, etc, mostrar que no está bueno.
  - Interpolación en espacios de tiempo muy largos.

5. Agrupación
  - Tanques

6. Normalizar valores
  - Variación por temperatura?
  - Variación por cantidad de agua?

Análisis
1. Gráfico de centros op seleccionados.
2. Patrones por industria (análisis que hizo Lau)
3. Descriptores estadísticos
  - Medidas de centralización
  - Medidas de dispersión
4. Manejo de inventarios por industria (idem 2?) (Lau) 
5. Correlación de features (analisis de Lau)
6. Derivada primera y derivada segunda
  - Derivada primera nos dice si el líquido sube o baja.
  - Derivada segunda, velocidad de crecimiento/decr? aceleración de crec/decr?
7. Consumo por unidad de tiempo por centro operativo.
  - Suma de diffs negativos absoluto? Quizás sobre media movil. Ver dispersión de media movil
8. Patrones de consumo por industria.
9. Descriptores estadísticos consumo general y por industria.
10. Media móvil de centros operativos por industria.
11. Media móvil de consumos de centros operativos por industria.



===================================

Análisis:
- Autocorrelación (correlacion para series temporales sobre los lags)

Plots:
- Time plot
- Seasonal plot
- Polar plot
- Lag plot (scatter?)

Predicciones naïve:
